# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

**Цель исследования:**
- спрогнозировать, уйдёт ли клиент из банка в ближайшее время или нет

**Задачи исследования:**
- загрузить и подготовить данные, поясняя порядок действий
- исследовать баланс классов
- обучить модель БЕЗ учёта дисбаланса, кратко описать выводы
- улучшить качество модели, учитывая дисбаланс классов
- обучить разные модели и найти лучшую, кратко описать выводы
- провести финальное тестирование

## Подготовка данных

1. Импортируем все нужные в работе библиотеки.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.preprocessing import OneHotEncoder

2. Подготовим данные и изучим их.

In [2]:
data = pd.read_csv('/datasets/Churn.csv')

2.1. В данных 10000 строк и 14 столбцов.

In [3]:
data.shape

(10000, 14)

2.2. Рассмотрим информацию о данных:

- В столбце 'Tenure' можно заметить пропуски
- Данные в столбцах 'RowNumber', 'CustomerId', 'CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited' - численные
- Данные в столбцах - 'Surname', 'Geography', 'Gender' - категориальные

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


2.3. Изучим данные - рассмотрим первые строки

In [5]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


3. Рассмотрим столбец 'Tenure' - количество лет, в течение которых человек является клиентом банка. Нам неизвестны эти значения. Т.к. пропущенные значения составляют больше 9%, то удалять их - не лучший вариант. Получается, можно "предсказать" эти значения по принципу, схожему с алгоритмом работы модели. Создавать новую модель для предсказания количества лет займёт много времени, поэтому можно выделить один "признак feature" для вычисления "целевого признака target" "вручную".

In [6]:
data['Tenure'].isna().sum() / 10000 * 100

9.09

3.1. Проведём мини-исследование. За feature возьмём признак возраста, за target - количество лет. Так как мы не будем обучать модель, лучший вариант - найти констатную модель. Есть два варианта определения константной модели - по среднему значению, как когда мы определяем константу для RMSE, или по медианному значению, как для определения константы для МАЕ.
Мы знаем, что MSE больше зависит от разброса значений, чем МАЕ, и не знаем, насколько сильный разброс у значений. Исходя из этого, вычислим медианные значения 'Tenure' по каждой группе возраста и заполним ими пропущенные значения.

In [7]:
for age in data['Age'].unique():
    median = data.loc[data['Age'] == age, 'Tenure'].median()
    data.loc[(data['Tenure'].isna()) & (data['Age'] == age), 'Tenure'] = median

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Проверим данные - в столбце 'Tenure' больше нет пропусков.

4. Удалим из данных столбцы:
- 'RowNumber' - номер строки в данных нам не нужен, т.к. они полностью повторяют индексы
- 'CustomerId' и 'Surname' - личные данные пользователей не нужны в исследовании, т.к. они никаким образом не помогут модели обучаться более эффективно

In [9]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

5. Объявим переменную numeric, убрав из списка исключённые столбцы и целевой признак 'Exited' т.к. это пригодится в исследовании.

In [10]:
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',\
           'IsActiveMember', 'EstimatedSalary']

6. Преобразуем столбцы с категориальными данными с помощью техники OHE.

In [11]:
data = pd.get_dummies(data, drop_first=True)

In [13]:
data.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0


7. Выделим features и target, разделим данные на тренировочную, валидационную и тестовую выборки. Тренировочная выборка составит 60% от всех данных, валидационная - 20%, тестовая - 20%.

In [14]:
features = data.drop('Exited', axis=1)
target = data['Exited']

In [15]:
features_train_valid, features_test, target_train_valid, target_test = \
train_test_split(features, target, test_size=0.2, random_state=12345)
features_train, features_valid, target_train, target_valid = \
train_test_split(features_train_valid, target_train_valid, test_size=0.25, random_state=12345)
#тестовая часть содержит 25% от оставшихся данных

7.1. Проверим размеры выборок

In [16]:
display(features_train.shape, target_train.shape)
display(features_valid.shape, target_valid.shape)
features_test.shape, target_test.shape

(6000, 11)

(6000,)

(2000, 11)

(2000,)

((2000, 11), (2000,))

8. Масштабируем численные признаки

In [18]:
pd.options.mode.chained_assignment = None

scaler = StandardScaler()
scaler.fit(features_train[numeric])

features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])



## Исследование задачи

1. Вычислим количества строк, имеющих положительные и отрицательные значения

In [19]:
target_train.value_counts()

0    4781
1    1219
Name: Exited, dtype: int64

In [20]:
4781 / 1219

3.922067268252666

Количество значений с отрицательным классом почти в 4 раза больше количества значений с положительным классом. Из этого следует:
вес класса «0» = 1.0
вес класса «1» = 3.9

2. Для исследования баланса классов построим модели без учёта дисбаланса классов и оценим их с помощью метрик F1 и AUC_ROC. Рассмотрим три вида моделей: логистическая регрессия, дерево решений и случайный лес.

1) Логистическая регрессия

In [21]:
model = LogisticRegression(solver='liblinear', random_state=12345)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid) #обучим модель

f1_valid = f1_score(target_valid, predicted_valid) #вычислим f1-меру

proba_valid = model.predict_proba(features_valid) #вычислим вероятности получения того или иного класса
proba_one_valid = proba_valid[:, 1] #вычислим вероятность получения положительного класса
auc_roc_valid = roc_auc_score(target_valid, proba_one_valid) #вычислим AUC-ROC

print('F1:', f1_valid)
print('AUC-ROC:', auc_roc_valid)

F1: 0.30131826741996237
AUC-ROC: 0.7702914710889355


F1-score составила 0.3 - низкое значение.\
AUC-ROC составила 0.77, что больше 0.5, как у константной модели.

F1-мера показала низкий результат, в то время как AUC-ROC показала значение выше среднего. Т.е. одна метрика показала низкое качество модели, в то время как другая метрика показала результат выше среднего. Из этого следует, что качество модели всегда нужно измерять разными метриками.

2) Дерево решений

In [22]:
#найдём глубину модели дерева решений, при которой F1-мера достигает своего максимума
best_depth = 0
best_f1 = 0
for depth in range(1, 16, 1):
        model = DecisionTreeClassifier(max_depth=depth, random_state=12345) 
        model.fit(features_train, target_train)
        predicted_valid = model.predict(features_valid)
        f1 = f1_score(target_valid, predicted_valid)
        if f1 > best_f1:
            best_f1 = f1
            best_depth = depth
print('Глубина:', best_depth)
print('F1:', best_f1)

Глубина: 7
F1: 0.5583596214511041


При глубине max_depth=7 F1-мера достигает своего максимума.

In [23]:
#вычислим F1-меру и AUC-ROC модели с глубиной 7, которая показала лучший результат
model = DecisionTreeClassifier(max_depth=7, random_state=12345)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid) #найдём предсказания модели на валидационной выборке
proba_valid = model.predict_proba(features_valid) #найдём вероятности положительного и отрицательного классов
proba_one_valid = proba_valid[:, 1] #найдём вероятность положительного класса для расчёта AUC-ROC

print('F1:', f1_score(target_valid, predicted_valid))
print('AUC-ROC:', roc_auc_score(target_valid, proba_one_valid))

F1: 0.5583596214511041
AUC-ROC: 0.8231010349393358


F1-мера составила 0.56 - на 0.2 больше, чем у модели логистической регрессии\
AUC-ROC составила 0.82 - на 0.05 больше, чем у модели логистической регрессии

Модель дерева решений показала результаты лучше, чем у модели логистической регрессии по обеим метрикам.

3) Случайный лес

In [24]:
%%time

#найдём глубину и количество деревьев модели случайного леса, при которой достигается максимальное значение F1-меры
best_est = 0
best_f1 = 0

for depth in range(1, 16, 1):
    for est in range(10, 101, 10):
        model = RandomForestClassifier(n_estimators=est, max_depth=depth, random_state=12345)
        model.fit(features_train, target_train)
        predicted_valid = model.predict(features_valid)
        f1 = f1_score(target_valid, predicted_valid)
        if f1 > best_f1:
            best_depth = depth
            best_f1 = f1
            best_est = est
print('Глубина:', best_depth)
print('Количество деревьев:', best_est)
print('F1:', best_f1)

Глубина: 13
Количество деревьев: 80
F1: 0.565008025682183
CPU times: user 40.1 s, sys: 209 ms, total: 40.3 s
Wall time: 40.3 s


Лучший результат метрики F1 дала модель с гиперпараметрами глубины 15 и количеством деревьев 20. Рассмотрим эту модель.

In [25]:
#вычислим F1-меру и AUC-ROC модели с глубиной 7, которая показала лучший результат
model = RandomForestClassifier(max_depth=15, n_estimators=20, random_state=12345)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid) #найдём предсказания модели на валидационной выборке
proba_valid = model.predict_proba(features_valid) #найдём вероятности положительного и отрицательного классов
proba_one_valid = proba_valid[:, 1] #найдём вероятность положительного класса для расчёта AUC-ROC

print('F1:', f1_score(target_valid, predicted_valid))
print('AUC-ROC:', roc_auc_score(target_valid, proba_one_valid))

F1: 0.5624012638230647
AUC-ROC: 0.8293017696175128


F1-мера составила 0.56 - почти такой же результат, как у модели дерева решений, но немного выше (больше на 0.004)\
AUC-ROC составила 0.82 - почти такой же результат, как у модели дерева решений, но немного выше (больше на 0.006)

Модель случайного леса показала почти такие же результаты, как модель дерева решений, но немного выше по обеим метрикам.

**Выводы:**
- Наименьший результат по метрикам F1 и AUC-ROC показала модель логистической регрессии. Значения метрик моделей дерева решений и случайного леса выше, чем у модели логистической регрессии, и они находятся примерно на одном уровне друг с другом. Лучший результаты по метрикам показала модель случайного леса.
- Стоит заметить, что значение F1 сильно изменилось при переходе от модели логистической регрессии к моделям дерева решений и случайного леса. Значения AUC-ROC оставались на примерно одном уровне на всех моделях.

## Борьба с дисбалансом

Есть два способа борьбы с дисбалансом: увеличение выборки (upsampling) и уменьшение выборки(downsampling). Рассмотрим оба.

1) Увеличение выборки

In [26]:
#разделим features и target на положительные и отрицательные по целевому признаку 
#и сохраним их в переменные features_zeros, features_ones, target_zeros, target_ones

features_zeros = features_train[target_train == 0]
features_ones = features_train[target_train == 1]
target_zeros = target_train[target_train == 0]
target_ones = target_train[target_train == 1]

#мы выяснили ранее, что отрицательных признаков в почти 4 раза больше, чем положительных
#объявим переменную repeat, в которой сохраним это значение
repeat = 4

features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat) #соединим таблицы, увеличив количество
target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat) #выборки с положительным целевым признаком

features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)
#перемешаем данные

In [27]:
target_upsampled.value_counts()

1    4876
0    4781
Name: Exited, dtype: int64

Рассмотрим модели на обновлённых данных. Изучим модели точно так же, как до увеличения выборки и проанализируем результаты.

1.1) Логистическая регрессия

In [28]:
model = LogisticRegression(solver='liblinear', random_state=12345)
model.fit(features_upsampled, target_upsampled) #обучим модель на обновлённых данных

#вычислим F1 и AUC-ROC
predicted_valid = model.predict(features_valid)
proba_valid = model.predict_proba(features_valid)
proba_one_valid = proba_valid[:, 1]

print('F1:', f1_score(target_valid, predicted_valid))
print('AUC-ROC:', roc_auc_score(target_valid, proba_one_valid))

F1: 0.476693051890941
AUC-ROC: 0.7725629014542559


Сравним значения метрик модели логистической регрессии до и после увеличения выборки:

- F1 модели логистической регрессии до увеличения выборки - 0.3, значение увеличилось на 0.17
- AUC-ROC модели логистической регрессии до увеличения выборки - 0.770, значение увеличилось на 0.002

Вывод: после увеличения выборки модель логистической регрессии даёт более высокие результаты

1.2) Дерево решений

In [29]:
best_depth = 0
best_f1 = 0

for depth in range(1, 16, 1):
    model = DecisionTreeClassifier(max_depth=depth, random_state=12345)
    model.fit(features_upsampled, target_upsampled)
    predicted_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predicted_valid)
    if f1 > best_f1:
        best_f1 = f1
        best_depth = depth
print('Глубина:', best_depth)
print('F1:', best_f1)

Глубина: 6
F1: 0.5587044534412956


Лучший результат метрики F1 дала модель со значением max_depth=6. Рассмотрим её.

In [30]:
model = DecisionTreeClassifier(max_depth=6, random_state=12345)
model.fit(features_upsampled, target_upsampled)

predicted_valid = model.predict(features_valid)
proba_valid = model.predict_proba(features_valid)
proba_one_valid = proba_valid[:, 1]

print('F1:', f1_score(target_valid, predicted_valid))
print('AUC-ROC:', roc_auc_score(target_valid, proba_one_valid))

F1: 0.5587044534412956
AUC-ROC: 0.809054407830633


Сравним значения метрик модели дерева решений до и после увеличения выборки:

F1 модели дерева решений до увеличения выборки - 0.56, значение увеличилось на 0.0004\
AUC-ROC модели дерева решений до увеличения выборки - 0.82, значение уменьшилось на 0.02

Вывод: после увеличения выборки модель дерева решений даёт более высокие результаты по метрике F1 и менее высокие по метрике AUC-ROC, но новые результаты очень близки к прежним.

1.3) Случайный лес

In [31]:
best_depth = 0
best_est = 0
best_f1 = 0

for depth in range(1, 16, 1):
    for est in range(10, 101, 10):
        model = RandomForestClassifier(max_depth=depth, n_estimators=est, random_state=12345)
        model.fit(features_upsampled, target_upsampled)
        predicted_valid = model.predict(features_valid)
        f1 = f1_score(target_valid, predicted_valid)
        if f1 > best_f1:
            best_f1 = f1
            best_est = est
            best_depth = depth
print('Глубина:', best_depth)
print('Количество деревьев:', best_est)
print('F1:', best_f1)

Глубина: 13
Количество деревьев: 40
F1: 0.6032540675844806


Лучший результат метрики F1 дала модель случайного леса с гиперпараметрами: глубиной max_depth=15 и количеством деревьев n_estimators=80. Рассмотрим её.

In [32]:
#обучим модель с глубиной max_depth=15 и количеством деревьев n_estimators=80 на обновлённых данных

model = RandomForestClassifier(max_depth=15, n_estimators=80, random_state=12345)
model.fit(features_upsampled, target_upsampled)

predicted_valid = model.predict(features_valid)
proba_valid = model.predict_proba(features_valid)
proba_valid_one = proba_valid[:, 1]

print('F1:', f1_score(target_valid, predicted_valid))
print('AUC-ROC:', roc_auc_score(target_valid, proba_valid_one))

F1: 0.5970548862115127
AUC-ROC: 0.8475630206685858


Сравним значения метрик модели случайного леса до и после увеличения выборки:

- F1 модели случайного леса до увеличения выборки - 0.56, значение увеличилось на 0.3
- AUC-ROC модели случайного леса до увеличения выборки - 0.82, значение увеличилось на 0.02

Вывод: значения метрик случайного леса улучшились после увеличения выборки.

**Значение F1 больше 0.59 достигнуто.**

2) Уменьшение выборки

Т.к. отрицательных значений почти в 4 раза больше, нужно уменьшить их количество в 4 раза, т.е. новая выборка будет составлять 0.25 часть от старой выборки.

In [33]:
#мы уже объявили нужные нам переменные features_zeros, features_ones, target_zeros, target_ones

#случайным образом отбросим значения отрицательного класса и оставим 0.25 часть от старой выборки
features_sample = features_zeros.sample(frac=0.25, random_state=12345)
target_sample = target_zeros.sample(frac=0.25, random_state=12345)

#соединим значения отрицательного и положительного классов
features_downsampled = pd.concat([features_sample] + [features_ones])
target_downsampled = pd.concat([target_sample] + [target_ones])

#перемешаем значения
features_downsampled, target_downsampled = shuffle(features_downsampled, target_downsampled, random_state=12345)

In [34]:
target_downsampled.value_counts()

1    1219
0    1195
Name: Exited, dtype: int64

Рассмотрим модели на обновлённых данных.

2.1) Логистическая регрессия

In [35]:
#обучим модель логистической регрессии на обновлённых данных
model = LogisticRegression(solver='liblinear', random_state=12345)
model.fit(features_downsampled, target_downsampled)

#рассчитаем F1 и AUC-ROC
predicted_valid = model.predict(features_valid)
proba_valid = model.predict_proba(features_valid)
proba_valid_one = proba_valid[:, 1]

print('F1:', f1_score(target_valid, predicted_valid))
print('AUC-ROC:', roc_auc_score(target_valid, proba_valid_one))

F1: 0.4750219106047327
AUC-ROC: 0.7737407390334738


F1 логистической регрессии без учёта дисбаланса - 0.30131826741996237\
F1 логистической регрессии после увеличения выборки - 0.476693051890941\
F1 логистической регрессии после уменьшения выборки - 0.4750219106047327

AUC-ROC логистической регрессии без учёта дисбаланса - 0.7702914710889355\
AUC-ROC логистической регрессии после увеличения выборки - 0.7725629014542559\
AUC-ROC логистической регрессии после уменьшения выборки - 0.7737407390334738

Наибольшее значение F1 модели логистической регрессии достигается при её обучении на увеличенной выборке, значение AUC-ROC - на уменьшенной. При этом значение F1 заметно увеличивается после балансировки классов. Значение AUC-ROC тоже увеличивается, но в меньшей степени.

2.2) Дерево решений

In [36]:
best_f1 = 0
best_depth = 0
#обучим модель на обновлённых данных и найдём значение глубины, при котором достигается наибольшее значение F1
for depth in range(1, 16, 1):
    model = DecisionTreeClassifier(max_depth=depth, random_state=12345)
    model.fit(features_downsampled, target_downsampled)
    predicted_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predicted_valid)
    if f1 > best_f1:
        best_f1 = f1
        best_depth = depth
print('Глубина:', best_depth)
print('F1:', best_f1)

Глубина: 7
F1: 0.5493482309124766


Лучшее значение F1-меры даёт модель дерева решений при значении максимальной глубины 7. Рассмотрим её.

In [37]:
#обучим модель дерева решений с глубиной 7 на обновлённых данных
model = DecisionTreeClassifier(max_depth=7, random_state=12345)
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)
proba_valid = model.predict_proba(features_valid)
proba_one_valid = proba_valid[:, 1]

print('F1:', f1_score(target_valid, predicted_valid))
print('AUC-ROC:', roc_auc_score(target_valid, proba_one_valid))

F1: 0.5493482309124766
AUC-ROC: 0.8143546769371137


F1 дерева решений без учёта дисбаланса - 0.5583596214511041\
F1 дерева решений после увеличения выборки - 0.5587044534412956\
F1 дерева решений после уменьшения выборки - 0.5493482309124766

AUC-ROC дерева решений без учёта дисбаланса - 0.8231010349393358\
AUC-ROC дерева решений после увеличения выборки - 0.809054407830633\
AUC-ROC дерева решений после уменьшения выборки - 0.8143546769371137

Наибольшее значение метрики F1 модели дерева решений достигается после увеличения выборки, наибольшее значение AUC-ROC достигается без учёта дисбаланса.

2.3) Случайный лес

In [38]:
best_f1 = 0
best_depth = 0
best_est = 0
#найдём значения гиперпараметров, при котором достигается наибольшее значение F1
for depth in range(1, 16, 1):
    for est in range(10, 101, 10):
        model = RandomForestClassifier(max_depth=depth, n_estimators=est, random_state=12345)
        model.fit(features_downsampled, target_downsampled)
        predicted_valid = model.predict(features_valid)
        f1 = f1_score(target_valid, predicted_valid)
        if f1 > best_f1:
            best_f1 = f1
            best_depth = depth
            best_est = est
print('Глубина:', best_depth)
print('Количество деревьев:', best_est)
print('F1:', best_f1)

Глубина: 9
Количество деревьев: 100
F1: 0.5700293829578844


Наибольшее значение F1 модели случайного леса достигается при максимальной глубине 9 и количестве деревьев 100. Рассмотрим эту модель.

In [39]:
#обучим модель случайного леса с глубиной 9 и количеством деревьев 100 на обновлённых данных
model = RandomForestClassifier(max_depth=9, n_estimators=100, random_state=12345)
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)
proba_valid = model.predict_proba(features_valid)
proba_one_valid = proba_valid[:, 1]

print('F1:', f1_score(target_valid, predicted_valid))
print('AUC-ROC:', roc_auc_score(target_valid, proba_one_valid))

F1: 0.5700293829578844
AUC-ROC: 0.8485167353076286


F1 случайного леса без учёта дисбаланса - 0.5624012638230647\
F1 случайного леса после увеличения выборки - 0.5986577181208055\
F1 случайного леса после уменьшения выборки - 0.5700293829578844

AUC-ROC случайного леса без учёта дисбаланса - 0.8293017696175128\
AUC-ROC случайного леса после увеличения выборки - 0.8475630206685858\
AUC-ROC случайного леса после уменьшения выборки - 0.8485167353076286

Наибольшее значение F1 модели случайного леса достигается после увеличения выборки, наибольшее значение AUC-ROC - после уменьшения выборки.

**Вывод:** таким образом, значение F1 больше 0.59 достигается только моделью случайного леса с гиперпараметрами max_depth=15 и количеством деревьев n_estimators=80 после увеличения выборки.

## Тестирование модели

Протестируем модель, которая дала лучший результат метрики F1.

In [40]:
#перезапишем в переменные модель с лучшими результатами
model = RandomForestClassifier(max_depth=15, n_estimators=80, random_state=12345)
model.fit(features_upsampled, target_upsampled)

#вычислим метрики F1 и AUC-ROC на тестовой выборке
predicted_test = model.predict(features_test)
proba_test = model.predict_proba(features_test)
proba_one_test = proba_test[:, 1]

print('F1:', f1_score(target_test, predicted_test))
print('AUC-ROC:', roc_auc_score(target_test, proba_one_test))

F1: 0.6133004926108374
AUC-ROC: 0.8560009290262643


Значение F1 на тестовой выборке составило 0.61 - больше, чем на валидационной на 0.02\
Значение AUC-ROC на тестовой выборке составило 0.85 - больше, чем на валидационной на 0.009

**ИТОГОВЫЙ ВЫВОД:**

В исследовании были рассмотрены модели логистической регрессии, дерева решений и случайного леса в трёх случаях: без учёта дисбаланса, после увеличения выборки, после уменьшения выборки. В моделях дерева решений и случайного леса были выявлены модели с гиперпараметрами, дающими наибольшие значения метрик F1 и AUC-ROC.

Стоит отметить, что при изменении гиперпараметров метрики F1 и AUC-ROC не всегда увеличивались или уменьшались синхронно - при изменении некоторых условий одна метрика увеличивалась, другая уменьшалась.

Наибольшие значения метрик дала модель случайного леса с глубиной 15 и количеством деревьев 80. Это единственная модель, значение F1-меры которой превысило 0.59. На тестовой проверке модель дала значения метрик ещё выше, чем на валидационной проверке.